In [ ]:
import trimesh
import open3d as o3d
import trimesh.exchange.stl

import numpy as np
import matplotlib.pyplot as plt

def plot_trimesh_edges_only(mesh, edge_color=[0, 0, 0, 128]):  # Black with 50% transparency
    """Plot only the edges with specified color and transparency"""
    
    # Create edge visualizations
    edges = mesh.edges_unique
    vertices = mesh.vertices
    
    # Create a collection of lines for each edge
    lines = []
    for edge in edges:
        lines.append(trimesh.path.entities.Line(edge))
    
    # Create colors for each entity (each line) with RGBA
    # Ensure we have 4 values (RGBA) for each color
    if len(edge_color) == 3:
        edge_color = list(edge_color) + [128]  # Add alpha if only RGB provided
    
    colors = np.tile(edge_color, (len(lines), 1))
    
    # Create a Path3D object
    path = trimesh.path.Path3D(
        entities=lines,
        vertices=vertices,
        colors=colors
    )
    
    scene = trimesh.Scene()
    scene.add_geometry(path)
    return scene

# Modify Irregular Spheres 

In [ ]:
irregular_spheres = trimesh.load_mesh('irregularSpheres_fromCAD.stl') #Packed_Spheres
irregular_spheres.show()

In [ ]:
irregular_spheres.is_watertight

In [ ]:
centroid = irregular_spheres.centroid
# Translate the mesh so that the centroid is at (0, 0, 0)
irregular_spheres.apply_translation(-centroid)

# Get bounding box corners
min_corner, max_corner = irregular_spheres.bounds  # shape (2, 3)

# Compute size along each axis (X, Y, Z)
size = max_corner - min_corner

print(f"Bounding box min corner: {min_corner}")
print(f"Bounding box max corner: {max_corner}")
print(f"Size (X x Y x Z): {size}")

In [ ]:
#offset the geometry
irregular_spheres.apply_translation((0.018,0.018,-0.04))
# print the new centroid
print(irregular_spheres.centroid)

In [ ]:
## determine appropiate grid spacing for Electric Field calculations

# Create mesh grid for exact sampling
WorldX, WorldY, WorldZ = 120, 105, 122+80
print(WorldX, WorldY, WorldZ )
stepsize = 10

x_array = np.arange(-WorldX/2, WorldX/2, stepsize)/1000
y_array = np.arange(-WorldY/2, WorldY/2, stepsize)/1000
z_array = np.arange(-WorldZ/2, WorldZ/2, stepsize)/1000

# Create 3D mesh grid
X, Y, Z = np.meshgrid(x_array, y_array, z_array, indexing='ij')

# Flatten the mesh grid to create sampling points
sampling_points = np.column_stack([X.ravel(), Y.ravel(), Z.ravel()])
print(len(sampling_points))
photoelectron_stopping_sites = trimesh.points.PointCloud(sampling_points, colors=[0, 0, 255, 255])

scene = plot_trimesh_edges_only(irregular_spheres, edge_color=[0, 0, 0, 128])
scene.add_geometry([photoelectron_stopping_sites])
scene.show()

In [ ]:
# Get ASCII STL string
ascii_stl_str = trimesh.exchange.stl.export_stl_ascii(mesh=irregular_spheres)

# Write it to a file
with open("irregularSpheres_fromPython.stl", "w") as f:
    f.write(ascii_stl_str)